In [ ]:
def conf(true, pred):
    df = pd.DataFrame()
    df['true'] = true
    df['pred'] = pred
    df['1'] = 1
    return df.pivot(index = 'true',)

In [40]:
df = pd.DataFrame()

In [41]:
df['1']  = [1,2,3,4,1,2,3,4]
df['2']  = [1,1,2,2,3,3,4,4]
df['3']  = [1,1,1,1,1,1,1,1]

In [42]:
df.pivot(index = '1', columns = '2', values='3')

2,1,2,3,4
1,,,,
1,1,NaN,1,NaN
2,1,NaN,1,NaN
3,NaN,1,NaN,1
4,NaN,1,NaN,1


## Homework 1

Author: Dmitry Petrov, teaching assistant (use to.dmitry.petrov@gmail.com for questions)

Prerequisites: sklearn, pandas, numpy, scipy, ipython/jupyter, xgboost, matplotlib/seaborn libraries for python.

Your report (as ipython-notebook preferebaly) should be prepared by June 17.

The letter with the report should be sent to these emails: to.dmitry.petrov@gmail.com, dmitrii.ignatov@gmail.com, and ivannnnz@gmail.com. The topic of the letter must have the following format [CS-ML2016m-HW1]-LAST NAME-FIRST NAME.

Max mark is 10.


### 0. Data aquisition

Get the training data from 'Walmart Recruiting: Trip Type Classification' competition:
https://www.kaggle.com/c/walmart-recruiting-trip-type-classification/data

Note 1: you can also pass this assignment if you will provide notebook with 0.55 log loss or better result on 10-fold cross-validation. 

Note 2: if this competition is your project, consider this assignment as first steps in it. 



In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pickle as pkl
%matplotlib inline
from tqdm import tqdm

In [2]:
train = pd.read_csv('./hw1/train.csv')

In [3]:
ss = pd.read_csv('./hw1/sample_submission.csv')

In [4]:
train.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,999,5,Friday,68113152929,-1,FINANCIAL SERVICES,1000
1,30,7,Friday,60538815980,1,SHOES,8931
2,30,7,Friday,7410811099,1,PERSONAL CARE,4504
3,26,8,Friday,2238403510,2,PAINT AND ACCESSORIES,3565
4,26,8,Friday,2006613744,2,PAINT AND ACCESSORIES,1017


In [5]:
ss.head()

,VisitNumber,TripType_3,TripType_4,TripType_5,TripType_6,TripType_7,TripType_8,TripType_9,TripType_12,TripType_14,...,TripType_36,TripType_37,TripType_38,TripType_39,TripType_40,TripType_41,TripType_42,TripType_43,TripType_44,TripType_999
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 1. Preliminary analysis

Give summary of train features. 

— Which of them are numerical and categorical? 

— What distributions do they have? Are distributions identical on test and train? 

— What features contain missing values? What's your guess why?

Note: Pandas library can be very handy.

In [6]:
train.columns

Index(['TripType', 'VisitNumber', 'Weekday', 'Upc', 'ScanCount',
       'DepartmentDescription', 'FinelineNumber'],
      dtype='object')

### 2. Data preprocessing and feature engineering

— Missing values. How do you choose to impute them? Why?

— String variables. Implement label and one-hot encoding. For what models it makes a difference? (Note: it may be better to use Pandas get_dummies method to one-hot encoder in sklearn).

— Combine observations by VisitNumber (Pandas groupby method can be very handy in that case). Make feature vector for each VisitNumber.  

— Split training data into even new_train and evaluation sets using train_test_split from sklearn. Fix random state for reproducibility of your results. 

— Choose cross-validation 5-fold split for your data (3-fold if your PC is not so fast). Do not forget to specify random state for your results reproducibility. 

### 3. KNN-classifier

— Try KNN classifier on the obtained features. Report accuracy and logarithmic loss (mean and std across the folds). How does this evaluation metrics change depending on neighbors number? Depending on distance metric? 

— Show normalized confusion matrix for the best obtained model performance on the evaluation set. How does it differ from cross-validation score?

In [13]:
from sklearn.neighbors import KNeighborsClassifier as KNN

In [14]:
import sklearn.cross_validation  as cv

In [18]:
cv_10 = cv.KFold(n = train.shape[0], n_folds = 10, shuffle = True, random_state = 42)

In [20]:
for i in cv_10:
    print(train.loc[i[0],'TripType'].nunique())

38
38
38
38
38
38
38
38
38
38


In [21]:
from sklearn.metrics import log_loss

In [22]:
train.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,999,5,Friday,68113152929,-1,FINANCIAL SERVICES,1000
1,30,7,Friday,60538815980,1,SHOES,8931
2,30,7,Friday,7410811099,1,PERSONAL CARE,4504
3,26,8,Friday,2238403510,2,PAINT AND ACCESSORIES,3565
4,26,8,Friday,2006613744,2,PAINT AND ACCESSORIES,1017


In [23]:
target = 'TripType'
features = ['VisitNumber', 'Weekday', 'Upc', 'ScanCount',
       'DepartmentDescription', 'FinelineNumber']
features_raw = ['VisitNumber', 'Upc', 'ScanCount', 'FinelineNumber']

In [26]:
for i in features_raw:
    if sum(pd.isnull(train[i])) > 0:
        train[i] = train[i].fillna(np.mean(train[i]))

In [32]:
for i in range(2,11):
    model = KNN(n_neighbors = i,n_jobs=-1)
    score = cv.cross_val_score(model,train.loc[:,features_raw],train.loc[:,target], cv = cv_10, verbose=10,scoring='log_loss')
    print(np.mean(score))

[CV] no parameters to be set .........................................
[CV] .............. no parameters to be set, score=-19.679493 -   9.9s
[CV] no parameters to be set .........................................
[CV] .............. no parameters to be set, score=-19.685479 -   9.5s
[CV] no parameters to be set .........................................
[CV] .............. no parameters to be set, score=-19.590008 -   9.1s
[CV] no parameters to be set .........................................
[CV] .............. no parameters to be set, score=-19.655886 -  10.7s

[Parallel(n_jobs=1)]: Done   1 tasks       | elapsed:   10.0s
[Parallel(n_jobs=1)]: Done   4 tasks       | elapsed:   39.9s



[CV] no parameters to be set .........................................
[CV] .............. no parameters to be set, score=-19.648031 -   9.7s
[CV] no parameters to be set .........................................
[CV] .............. no parameters to be set, score=-19.626806 -  10.5s
[CV] no parameters to be set .........................................
[CV] .............. no parameters to be set, score=-19.640585 -  10.8s
[CV] no parameters to be set .........................................
[CV] .............. no parameters to be set, score=-19.689972 -  10.7s
[CV] no parameters to be set .........................................
[CV] .............. no parameters to be set, score=-19.721208 -  11.5s
[CV] no parameters to be set .........................................
[CV] .............. no parameters to be set, score=-19.700253 -  10.3s

[Parallel(n_jobs=1)]: Done   7 tasks       | elapsed:  1.2min
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:  1.7min finished



-19.6637720735
[CV] no parameters to be set .........................................
[CV] .............. no parameters to be set, score=-17.284536 -  10.2s
[CV] no parameters to be set .........................................
[CV] .............. no parameters to be set, score=-17.299224 -   9.7s
[CV] no parameters to be set .........................................
[CV] .............. no parameters to be set, score=-17.179927 -   9.3s
[CV] no parameters to be set .........................................
[CV] .............. no parameters to be set, score=-17.249147 -  10.8s

...


-10.8616430558
[CV] no parameters to be set .........................................
[CV] .............. no parameters to be set, score=-10.414034 -  11.5s
[CV] no parameters to be set .........................................
[CV] .............. no parameters to be set, score=-10.323753 -  12.1s
[CV] no parameters to be set .........................................
[CV] .............. no parameters to be set, score=-10.213013 -  12.3s
[CV] no parameters to be set .........................................
[CV] .............. no parameters to be set, score=-10.378059 -  12.6s

[Parallel(n_jobs=1)]: Done   1 tasks       | elapsed:   11.6s
[Parallel(n_jobs=1)]: Done   4 tasks       | elapsed:   49.0s



[CV] no parameters to be set .........................................
[CV] .............. no parameters to be set, score=-10.388939 -51.9min
[CV] no parameters to be set .........................................
[CV] .............. no parameters to be set, score=-10.293692 -   7.3s
[CV] no parameters to be set .........................................
[CV] .............. no parameters to be set, score=-10.298436 -   7.3s
[CV] no parameters to be set .........................................
[CV] .............. no parameters to be set, score=-10.397795 -   4.4s
[CV] no parameters to be set .........................................
[CV] .............. no parameters to be set, score=-10.422401 -   4.1s
[CV] no parameters to be set .........................................
[CV] .............. no parameters to be set, score=-10.419146 -   3.8s

[Parallel(n_jobs=1)]: Done   7 tasks       | elapsed: 53.0min
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed: 53.2min finished



-10.3549266742


### 4. Linear models 

— Try logistic regression and linear SVM on your features. Report accuracy and log loss. Compare it to KNN-classifier. How does performance differ whether you use data scaling or not? How does performance differ if you use on-hot or label encoded categorical features?

— Regularization. Play with l1 and l2 regularization parameters. Plot log loss score depending on different C values for l1 and l2 regularizations. What regualarization do you choose with which parameter? Why? 

— Show normalized confusion matrix for the best obtained model performance on the evaluation set. How does it differ from cross-validation score?

### 5. Tree-based models

— Try vanilla Random Forest, AdaBoost, GBDT from sklearn and GBDT from xgboost. Report cross-validation log loss (mean and std) for each of these models.

— Random Forest. Find best parameters for the Random Forest using grid search on cross-validation. Which of them are the most important? Report result on evaluation test. 

— Adaboost. Find the best values of parameters for the AdaBoost using grid search on cross-validation. Which of them are the most important? Plot learning curve (log loss on each conscutive iteration) for the best AdaBoost on new_train and evaluation sets. Note: staged_predict_proba method can be very useful. 

— GBDT. Compare speed and performance of sklearn/xgboost GBDT implementation with similar parameters. Which implementation do your choose? Do not forget to utilise all of your CPUs/threads to speed up calculations. Find best parameters for the GBDT using grid search on cross-validation. What are the most important? Plot learning curve (log loss on each consecutive iteration) for the best GBDT on new_train and evaluation sets. Note: staged_predict_proba method can be very useful for sklearn and evals_result method for xgboost.


### 6. Model comaprison and combination

— Compare performance of all used models in terms of accuracy and log loss on evaluation. What is the best single model performance?

— (optional) Try to imporve your single model performnace with weighted combination of other models predictions. Note: sklearn VotingClassifier with 'softmax' option can be useful. What is the best model performance on evaluation?